<p style="font-weight:bold;"> <span style="font-size: 36px"> Ifrs Variable Exporter </span> </p>

# Infrastructure and Configuration

## Initialize Workspace

In [0]:
#!eval-notebook "InitSystemorphToMemoryForTesting"
//#!eval-notebook "../../PracticalUseCases/SingleVsMultipleCsmSwitch/CsmSwitchDataImport"

## Extra Imports

In [0]:
ProjectionCount = 5;
var activity1 = await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2020_12_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();
var activity2 = await Import.FromFile("../../Files/TransactionalData/NominalCashflows_CH_2021_3_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();
ProjectionCount = 1;

## Configurations

In [0]:
//var pathToBm = "./Data/ReportBenchmarks/";

In [0]:
int roundingDigits = - (int)Math.Log10(BenchmarkPrecision) + 1;

# Defining Scope for the IfrsVariables

## Defining the Import Args

In [0]:
var exportScope = new ImportArgs[]{
    new ImportArgs ("CH", 2020, 12, (Periodicity)default, null, null),
    new ImportArgs ("CH", 2021, 3,  (Periodicity)default, null, null),
    new ImportArgs ("CH", 2020, 12, (Periodicity)default, "MTUP10pct", null)
    };

# Gathering Data

In [0]:
var args = new ImportArgs ("CH", 2021, 3, (Periodicity)default, null, null);
args

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = args.ReportingNode, Year = args.Year, Month = args.Month, Scenario = args.Scenario});
var ifrsVariables = (await DataSource.Query<IfrsVariable>().ToArrayAsync())
            .Where(x => x.DataNode == "DTP1.1") //Extra Filter
            .Select(v => v with {Values = v.Values.Select(x => Math.Round(x, roundingDigits)).ToArray()})
            .OrderBy(v => v.DataNode).ThenBy(v => v.AccidentYear).ThenBy(v => v.EstimateType).ThenBy(v => v.EconomicBasis).ThenBy(v => v.AmountType).ThenBy(v => (v.AocType, v.Novelty))
            .ThenBy(v => v.AocType)
            .ToArray();

# Export

## Export SimpleValue

In [0]:
string fileNamePrefix = "SimpleValue";

In [0]:
// await Export.ToCsv(fileNamePrefix + "_" + args.ReportingNode + "_" + args.Year.ToString() + "_" + args.Month.ToString() + "_" + args.Scenario)
//                                     .WithTable<ImportArgs>(tableConfig => tableConfig
//                                                                     .AtBeginning()
//                                                                     .WithName(Main)
//                                                                     .WithSource(source => args.RepeatOnce().AsQueryable())
//                                                                     .WithColumn(x => x.Periodicity, x => x.Delete())
//                                                                     .WithColumn(x => x.ImportFormat, x => x.Delete())
//                                                     )
//                                     .WithTable<IfrsVariable>(tableConfig => tableConfig
//                                                                   .WithSource(source => ifrsVariables.AsQueryable())
//                                                                   .WithName(fileNamePrefix)
//                                                                   .WithColumn(x => x.Partition, x => x.Delete())
//                                                                   .WithColumn(x => x.Id, x => x.Delete())
//                                                                  )
//                                     .WithSource(DataSource)
//                                     .ExecuteAsync()    

## Export by Estimate Type

In [0]:
string benchmarkFileNamePrefix = "BM";

In [0]:
string benchmarkFileNameSuffix = "Projection_DTP";

In [0]:
var estimateType = EstimateTypes.BE;

In [0]:
var filename = benchmarkFileNamePrefix + "_" + args.ReportingNode +"_" + args.Year.ToString() + "_" + args.Month.ToString() + "_" + args.Scenario + "_" + estimateType + (string.IsNullOrWhiteSpace(benchmarkFileNameSuffix) ? string.Empty : "_" + benchmarkFileNameSuffix);
await Export.ToCsv(filename)
    .WithTable<ImportArgs>(tableConfig => tableConfig
                                    .AtBeginning()
                                    .WithName(Main)
                                    .WithSource(source => args.RepeatOnce().AsQueryable())
                                    .WithColumn(x => x.Periodicity, x => x.Delete())
                                    .WithColumn(x => x.ImportFormat, x => x.Delete())
                    )
    .WithTable<IfrsVariable>(tableConfig => tableConfig
    .WithSource(source => ifrsVariables.Where(x => x.EstimateType == estimateType).AsQueryable())
                                  .WithName(estimateType)
                                  .WithColumn(x => x.Partition, x => x.Delete())
                                  .WithColumn(x => x.Id, x => x.Delete())
                                 )
    .WithSource(DataSource)
    .ExecuteAsync()                         

In [0]:
ifrsVariables